In [ ]:
from dotenv import load_dotenv

load_dotenv()

### STT


In [ ]:
import pyaudio

p = pyaudio.PyAudio()

info = p.get_default_host_api_info()
for i in range(info.get("deviceCount")):
    if (p.get_device_info_by_host_api_device_index(0, i).get("maxInputChannels")) > 0:
        print(
            "Input Device id ",
            i,
            " - ",
            p.get_device_info_by_host_api_device_index(0, i).get("name"),
        )

p.terminate()

In [ ]:
import wave

import pyaudio

p = pyaudio.PyAudio()

DEVICE_INDEX = 4
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 5

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    input_device_index=DEVICE_INDEX,
    frames_per_buffer=CHUNK,
)

print("recording")
wf = wave.open("./data/recording.wav", "wb")
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)

for _ in range(int(RATE / CHUNK * RECORD_SECONDS)):
    wf.writeframes(stream.read(CHUNK))

wf.close()
stream.stop_stream()
stream.close()
p.terminate()
print("done recording")

# LEFT HERE
# TODO: feed the recording into transcription model

In [ ]:
from openai import OpenAI

openai = OpenAI()

with open("./data/recording.wav", "rb") as f:
    response = openai.audio.transcriptions.create(model="whisper-1", file=f)
    print(response)

### TTS


In [ ]:
import time

import pyaudio
from openai import OpenAI

openai = OpenAI()

player_stream = pyaudio.PyAudio().open(
    format=pyaudio.paInt16, channels=1, rate=24000, output=True
)

start_time = time.time()

with openai.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="alloy",
    response_format="pcm",  # similar to WAV, but without a header chunk at the start.
    input="""I see skies of blue and clouds of white 
            The bright blessed days, the dark sacred nights 
            And I think to myself 
            What a wonderful world""",
) as response:
    print(f"Time to first byte: {int((time.time() - start_time) * 1000)}ms")
    for chunk in response.iter_bytes(chunk_size=1024):
        player_stream.write(chunk)

print(f"Done in {int((time.time() - start_time) * 1000)}ms.")

In [ ]:
import time

import pyaudio
from openai import OpenAI, Stream

openai = OpenAI()


def say(input: str | Stream):
    if isinstance(input, str):
        say_string(input)
    else:
        say_stream(input)


def say_stream(input: Stream):
    sentence_terminators = [".", "?", "!"]
    sentence = ""
    for chunk in input:
        content = chunk.choices[0].delta.content
        if content:
            for char in content:
                if (
                    char not in sentence_terminators
                    and sentence[-1:] in sentence_terminators
                ):
                    # End of sentence
                    # This blocks until playback of the sentence is done
                    # TODO: continue reading input stream while playing
                    say_string(sentence.strip())
                    sentence = char
                else:
                    sentence += char
    if sentence:
        say_string(sentence.strip())


def say_string(input: str):
    print(f"Saying: {input}")

    p = pyaudio.PyAudio()
    stream = p.open(
        format=pyaudio.paInt16,
        channels=1,
        rate=24000,
        output=True,
    )

    start_time = time.time()
    with openai.audio.speech.with_streaming_response.create(
        model="tts-1", voice="alloy", response_format="pcm", input=input
    ) as response:
        print(f"Time to first byte: {int((time.time() - start_time) * 1000)}ms")
        for chunk in response.iter_bytes(chunk_size=1024):
            stream.write(chunk)

        print(f"Done in {int((time.time() - start_time) * 1000)}ms.")

In [ ]:
say("""\
I see skies of blue and clouds of white 
The bright blessed days, the dark sacred nights 
And I think to myself 
What a wonderful world"
""")

In [ ]:
import time

from openai import OpenAI

openai = OpenAI()

start_time = time.time()
with openai.audio.speech.with_streaming_response.create(
    model="tts-1", voice="alloy", response_format="wav", input="Hello, world!"
) as response:
    num_chunks = 0
    for chunk in response.iter_bytes():
        print(f"{time.time() - start_time:.3f} - {len(chunk)}")

In [ ]:
from openai import OpenAI

openai = OpenAI()

response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Write a short uplifiting poem"}
    ],
    stream=True
)

say(response)